In [1]:
!beet describe genre

┌────────────────┬───────────────────────────┐
│ Name           │                     Value │
╞════════════════╪═══════════════════════════╡
│ Field name     │                     genre │
├────────────────┼───────────────────────────┤
│ Field type     │ beets.dbcore.types.String │
├────────────────┼───────────────────────────┤
│ Auto type      │                      None │
├────────────────┼───────────────────────────┤
│ Count          │                      6796 │
├────────────────┼───────────────────────────┤
│ Unique         │                       213 │
├────────────────┼───────────────────────────┤
│ Most frequent  │                 Jazz(732) │
├────────────────┼───────────────────────────┤
│ Least frequent │      Baroque;Classical(1) │
├────────────────┼───────────────────────────┤
│ Empty          │                       401 │
└────────────────┴───────────────────────────┘
describe: Unique element histogram
Jazz                                               [732]  ██████████████

In [111]:
import subprocess
from subprocess import PIPE
import os
import pandas as pd
import random
from getpass import getpass

#get pass
password = getpass('Password pliz: ')

# define mp3 player max space
MAX_SPACE = 50 * 1024**2
player_path = '/home/matias/MECHEN'

# get not listened to albums and their paths
album_query = subprocess.run('beet ls -ap status:0', shell=True, capture_output=True)
album_paths = album_query.stdout.decode('ascii').splitlines()
short_paths = [i.replace('/home/matias/music/','') for i in album_paths]
print(f"{len(album_paths)} not listened to")

# calculate sizes
sizes = [subprocess.run(['du','-sk',i],capture_output=True) for i in album_paths]
sizes = [int(i.stdout.decode('ascii').split('\t')[0]) for i in sizes]
print(f"Total size equal to {round(sum(sizes)/1024/1024,2)} GB")

# get artists
artist = [i.split('/')[4] for i in album_paths]

# create df
df = pd.DataFrame({'path' : album_paths,
                   'short_path' : short_paths,
                   'artist' : artist,
                   'size' : sizes})

def drop_random_artist(df):
    to_drop = random.choice(list(set(df.artist)))
    print(to_drop)
    return df[df['artist']!=to_drop]

def df_too_big(df):
    return df['size'].sum() > MAX_SPACE

while df_too_big(df):
    print(f'Too much data, removing:')
    df = drop_random_artist(df)

# get albums already on device
artists_on_device = [i for i in os.listdir(player_path) if i != 'ALBUM.PIC']
albums_on_device = [[f"{i}/{b}" for b in os.listdir(os.path.join(player_path,i))]
                    for i in artists_on_device]

albums_on_device = [i for j in albums_on_device for i in j]

# remove unecessary albums
for i in albums_on_device:
    if i not in df['short_path'].to_list():
        proc = subprocess.Popen(['sudo','-S','rm','-rf',os.path.join(player_path,i)],
                                stdin=PIPE, stdout=PIPE, stderr=PIPE)
        proc.communicate(password.encode())

# remove empty folder
for i in os.listdir(player_path):
    p = os.path.join(player_path, i)
    if os.path.isdir(p):
        if not os.listdir(p):
            proc = subprocess.Popen(['sudo','-S','rm','-rf',p],
                                stdin=PIPE, stdout=PIPE, stderr=PIPE)
            proc.communicate(password.encode())

#generate artist and album folders
print('Creating folders...')
for i in df['short_path'].to_list():
    proc = subprocess.Popen(['sudo','-S','mkdir','-p',os.path.join(player_path,i)],
                                stdin=PIPE, stdout=PIPE, stderr=PIPE)
    proc.communicate(password.encode())

# rsync everything
nb_albums = len(df)
for n,i in enumerate(df['short_path'].to_list()):
    print(f"Rsyncing {i} ({n+1}/{nb_albums})")
    proc = subprocess.Popen(['sudo','-S','rsync','-rvu', '--delete',
                             os.path.join('/home/matias/music',i)+'/', #important for rsync to work properly
                             os.path.join(player_path,i)],
                                stdin=PIPE, stdout=PIPE, stderr=PIPE)
    proc.communicate(password.encode())

Password pliz: ········
344 not listened to
Total size equal to 89.09 GB
Too much data, removing:
Dexter Gordon
Too much data, removing:
Lydia Kavina
Too much data, removing:
Penny Penny
Too much data, removing:
Pamoja
Too much data, removing:
The Sun Ra Arkestra
Too much data, removing:
Robert Glasper
Too much data, removing:
William DeVaughn
Too much data, removing:
William Basinski
Too much data, removing:
Alva Noto
Too much data, removing:
Chicha Libre
Too much data, removing:
Nicolas Jaar
Too much data, removing:
Tasavallan Presidentti
Too much data, removing:
Abbey Lincoln
Too much data, removing:
Brenda Fassie
Too much data, removing:
John Cage
Too much data, removing:
Alva Noto + Ryuichi Sakamoto
Too much data, removing:
Flying Lotus
Too much data, removing:
Anatoly Lyadov; BBC Philharmonic, Vassily Sinaisky
Too much data, removing:
Jackie Wilson
Too much data, removing:
Fiona Apple
Too much data, removing:
The Clifford Brown-Max Roach Quintet
Too much data, removing:
The Wynto

Rsyncing Future/FUTURE (60/190)
Rsyncing The Future Sound of London/Lifeforms (61/190)
Rsyncing Marvin Gaye/I Heard It Through the Grapevine! (62/190)
Rsyncing Stan Getz & Joao Gilberto/Getz_Gilberto (63/190)
Rsyncing Global Communication/76_14 (64/190)
Rsyncing Grant Green/Idle Moments (65/190)
Rsyncing Greg Haines/Digressions (66/190)
Rsyncing Tigran Hamasyan/Mockroot (67/190)
Rsyncing Tigran Hamasyan/StandArt (68/190)
Rsyncing Herbie Hancock with Jaco Pastorius/Ivanhoe Theatre (69/190)
Rsyncing Herbie Hancock, Michael Brecker & Roy Hargrove/Directions in Music_ Live at Massey Hall (70/190)
Rsyncing John Handy/Carnival (71/190)
Rsyncing Tim Hecker/Love Streams (72/190)
Rsyncing Herbie Hancock, Ron Carter, Billy Cobham/World Of Rhythm Live (73/190)
Rsyncing Andrew Hill/Point of Departure (74/190)
Rsyncing The Hives/The Black and White Album (75/190)
Rsyncing Ari Hoenig, Nitai Hershkovits & Or Bareket/Conner's Days (76/190)
Rsyncing Freddie Hubbard/Red Clay (77/190)
Rsyncing Freddie Hu

In [4]:
import subprocess as sp
from subprocess import PIPE

sp.Popen(['echo','\n'.join(['a','b','c']),'|','dmenu','-l','30'],stdin=PIPE, stdout=PIPE, stderr=PIPE)